In [23]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.impute import SimpleImputer

In [24]:
def load_data():
    df_train = pd.read_csv('train.csv')
    df_test = pd.read_csv('test.csv')
    df_data_dict = pd.read_excel('data dictionary.xlsx')
    return df_train, df_test, df_data_dict

In [25]:
def preprocess_data(df_train, df_test):
    # Définir les colonnes à sélectionner pour l'entraînement du modèle
    features = ['Year', 'Kilometers_Driven', 'Mileage', 'Engine', 'Power', 'Seats', 'Fuel_Type', 'Transmission', 'Owner_Type']

    # Vectoriser la colonne "Name" en utilisant TF-IDF
    vectorizer = TfidfVectorizer()
    X_train_name = vectorizer.fit_transform(df_train['Name'])
    X_test_name = vectorizer.transform(df_test['Name'])

    # Appliquer le clustering k-means
    kmeans = KMeans(n_clusters=10)
    kmeans.fit(X_train_name)

    # Ajouter les étiquettes de cluster à la colonne "Name"
    df_train['Name_Cluster'] = kmeans.labels_
    df_test['Name_Cluster'] = kmeans.predict(X_test_name)

    # Nettoyer et préparer les données
    df_train['Engine'] = df_train['Engine'].str.replace(' CC', '', regex=False)
    df_train['Mileage'] = df_train['Mileage'].str.replace(' kmpl| km/kg', '', regex=True).astype(float)
    df_train['Power'] = df_train['Power'].str.replace(' bhp', '').replace('null', np.nan).astype(float)

    df_test['Engine'] = df_test['Engine'].str.replace(' CC', '', regex=False)
    df_test['Mileage'] = df_test['Mileage'].str.replace(' kmpl| km/kg', '', regex=True).astype(float)
    df_test['Power'] = df_test['Power'].str.replace(' bhp', '').replace('null', np.nan).astype(float)

    # Supprimer la colonne New_Price
    df_train = df_train.drop(['New_Price'], axis=1)
    df_test = df_test.drop(['New_Price'], axis=1)

    # Séparer les données d'entraînement en features et cible
    X_train = df_train[features + ['Name_Cluster']]
    y_train = df_train['Price']

    # Séparer les données de test en features seulement
    X_test = df_test[features + ['Name_Cluster']]

    return X_train, y_train, X_test


In [26]:
def create_pipeline():
    num_cols = ['Year', 'Kilometers_Driven', 'Mileage', 'Engine', 'Power', 'Seats']
    cat_cols = ['Fuel_Type', 'Transmission', 'Owner_Type', 'Name_Cluster']

    # Définir les transformations pour les colonnes numériques et catégorielles
    num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])

    cat_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Combiner les transformations en une seule étape de prétraitement
    preprocessor = ColumnTransformer([
        ('num', num_pipeline, num_cols),
        ('cat', cat_pipeline, cat_cols)
    ])

    # Définir le modèle de régression
    model = GradientBoostingRegressor(random_state=42)

    # Créer la pipeline
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    return pipeline

In [27]:
def train_and_evaluate_model(X, y):
    pipeline = create_pipeline()

    # Diviser les données d'entraînement en ensembles d'entraînement et de validation
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Entraîner le modèle sur les données d'entraînement
    pipeline.fit(X_train, y_train)

    # Évaluer les performances du modèle sur les données de validation
    score = pipeline.score(X_val, y_val)
    print('Score de la pipeline sur les données de validation :', score)

    # Faire des prédictions sur les données de validation
    y_pred = pipeline.predict(X_val)

    # Coefficient de détermination (R²)
    r2 = r2_score(y_val, y_pred)
    print("R² :", r2)

    # Erreur moyenne absolue (MAE)
    mae = mean_absolute_error(y_val, y_pred)
    print("MAE :", mae)

    # Erreur quadratique moyenne (MSE)
    mse = mean_squared_error(y_val, y_pred)
    print("MSE :", mse)

    # Erreur quadratique moyenne racine (RMSE)
    rmse = np.sqrt(mse)
    print("RMSE :", rmse)

    return pipeline

In [28]:
def predict_test_data(pipeline, df_test):


    features = ['Year', 'Kilometers_Driven', 'Mileage', 'Engine', 'Power', 'Seats', 'Fuel_Type', 'Transmission', 'Owner_Type']

    # Transformer les données de test à l'aide du pipeline
    df_test = df_test[features + ['Name_Cluster']]
    X_test = pipeline.named_steps['preprocessor'].transform(df_test)

    # Faire des prédictions sur les données de test
    y_pred = pipeline.named_steps['model'].predict(X_test)

    return y_pred

In [29]:
def add_predictions_to_df(df_test, y_pred):
    # Ajouter les prédictions à df_test
    df_test['Price'] = y_pred

    # Convertir les prix en unités de lakhs en roupies indiennes
    conversion_rate = 87
    lakh_conversion = 100000
    df_test['Price_Rupie'] = df_test['Price'] * lakh_conversion

    # Convertir les prix en euros
    df_test['Price_EUR'] = df_test['Price_Rupie'] / conversion_rate

    df_test['Price_EUR'] = df_test['Price_EUR'].round(2)

    return df_test


In [30]:
# Charger les données
df_train, df_test, df_data_dict = load_data()

# Prétraiter les données
X, y, df_test = preprocess_data(df_train, df_test)

# Entraîner et évaluer le modèle
pipeline = train_and_evaluate_model(X, y)

# Faire des prédictions sur les données de test
y_pred = predict_test_data(pipeline, df_test)

df_test = add_predictions_to_df(df_test, y_pred)

df_test.head()

Score de la pipeline sur les données de validation : 0.8570384342772641
R² : 0.8570384342772641
MAE : 2.0024260976763304
MSE : 17.592777638100824
RMSE : 4.194374522869986


,Year,Kilometers_Driven,Mileage,Engine,Power,Seats,Fuel_Type,Transmission,Owner_Type,Name_Cluster,Price,Price_Rupie,Price_EUR
0,2014,40929,32.26,998,58.20,4.0,CNG,Manual,First,8,3.266582,3.266582e+05,3754.69
1,2013,54493,24.70,796,47.30,5.0,Petrol,Manual,Second,8,2.595719,2.595719e+05,2983.58
2,2017,34000,13.68,2393,147.80,7.0,Diesel,Manual,First,6,18.214689,1.821469e+06,20936.42
3,2012,139000,23.59,1364,NaN,5.0,Diesel,Manual,First,6,4.977179,4.977179e+05,5720.90
4,2014,29000,18.50,1197,82.85,5.0,Petrol,Manual,First,7,4.551470,4.551470e+05,5231.58
